In [1]:
import numpy as np
import sys
import os
sys.path.append(os.getcwd()+'/src/')
from LCD import *

In [ ]:
dist, n = "normal", 5000
d = 4

In [ ]:
seed = 1

In [ ]:
if dist in ("normal","laplace"):
    sample = Sample(n=n,d=4,dist=dist,random_state=seed)
elif dist == "stock":
    # An example of customized data set
    import pandas as pd
    df = pd.read_csv('stock_returns_pool1.csv')
    X = df.iloc[-10000:,1:].values
    X/=np.std(X,axis=0)
    sample = Sample(X=X,dist="stock",random_state=1)

In [ ]:
N0dict = {"normal":[12,16,20,24], "laplace":[18,22,26,30], "stock":[14,18,22,26]}

In [ ]:
for N0 in N0dict[dist]:
    sample.generate_feas_grid_mat(method ="uniform",N0=N0)
    print(N0, sample.N)

In [ ]:
N0list = N0dict[dist]
sample.generate_feas_grid_mat(method ="uniform",N0=N0list[0])
nclcd1 = NCLCD(sample,K=10,maxIters = 100,tol=1e-4, plot=False)
nclcd1.subgradient()
init_kwargs = dict()
init_kwargs["init_suffix"] = "_nc_init_10_100_1e-04_%d"%(N0list[0])
init_kwargs["phi_start"] = nclcd1.iterates[nclcd1.disc_arg_min].phi
nclcd1.calcExactFinalObjective(nclcd1.iterates[nclcd1.disc_arg_min].phi)

In [ ]:
random_state = 42

In [ ]:
## RS-DI
sigma = 1e-4
N0list = N0dict[dist]
clcd = ConvexLCD(sample,maxIters = 128,maxtime=28800,random_state=random_state, evaluation=False,plot=False)
phi_hat = clcd.randomized_smoothing_approx(init_method="given",init_kwargs=init_kwargs,sigma=sigma, N0list=N0list,grid_method ="uniform")

In [ ]:
## RS-RI
sigma = 1e-4
Nlist = [10000,20000,40000,80000]
clcd = ConvexLCD(sample,maxIters = 128,maxtime=28800,random_state=random_state, evaluation=False,plot=False)
phi_hat = clcd.randomized_smoothing_stoch(init_method="given",init_kwargs=init_kwargs,sigma=sigma, Nlist= Nlist)

In [ ]:
## NS-DI
sigma = 1e-3
N0list = N0dict[dist]
clcd = ConvexLCD(sample,maxIters = 128,maxtime=36000,random_state=random_state, evaluation=False, plot=False)
phi_hat = clcd.nesterov_smoothing_approx(init_method="given",init_kwargs=init_kwargs,sigma=sigma, N0list= N0list,grid_method ="uniform")

In [ ]:
## NS-RI
sigma = 1e-3
N0list = [10000,20000,40000,80000]
clcd = ConvexLCD(sample,maxIters = 128,maxtime=36000,random_state=random_state, evaluation=False, plot=False)
phi_hat = clcd.randomized_smoothing_stoch(init_method="given",init_kwargs=init_kwargs,sigma=sigma, Nlist= Nlist)